In [1]:
import json
import logging
import pickle
from pathlib import Path

import chromadb
import numpy as np
from tqdm import tqdm, trange

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Initialization

In [9]:
# directory, where Chroma will store collections
CHROMADB_STORAGE_FOLDER = Path("/home/deniskirbaba/Documents/legal-ai/storage")

# directory with data, used to init legal practice collection (i.e. documents, embeddings, metadata)
LEGAL_PRACTICES_DATA_FOLDER = Path("/home/deniskirbaba/Documents/legal-ai/practices_data")


def squeeze_meta(meta: dict) -> dict:
    """
    Convert metadatas lists to strings
    """
    meta["codex"] = "".join(meta["codex"])
    meta["theme"] = "\n".join(meta["theme"])
    return meta


def init_legal_practices_db():
    """
    Initializes ChromaDB vector storage with legal practices docs.
    """
    persistent_client = chromadb.PersistentClient(path=str(CHROMADB_STORAGE_FOLDER))
    collection = persistent_client.create_collection(
        name="legal_practices", get_or_create=True, metadata={"hnsw:space": "cosine"}
    )

    # Load doc uids
    logging.info("Start loading doc uids.")
    with open(LEGAL_PRACTICES_DATA_FOLDER / "summaries_emb_uids.pkl", "rb") as f:
        doc_uids = pickle.load(f)
    logging.info("Successfully load doc uids.")

    # Load embeddings
    logging.info("Start loading embeddings.")
    embs = np.load(LEGAL_PRACTICES_DATA_FOLDER / "summaries_emb.npy")
    logging.info("Successfully load embeddings.")

    # Load metadatas (dict with docs uid and metadata (codex and theme))
    logging.info("Start loading metadatas.")
    with open(LEGAL_PRACTICES_DATA_FOLDER / "doc2meta.json", "r") as f:
        metadatas = json.load(f)
    logging.info("Successfully load metadatas.")

    # Load summaries (actual docs)
    logging.info("Start loading summaries.")
    summaries_path = LEGAL_PRACTICES_DATA_FOLDER / "summaries"
    summaries = {}
    for summary_path in tqdm(summaries_path.iterdir()):
        with open(summary_path, "r") as f:
            summaries[summary_path.stem] = f.read()
    logging.info("Successfully load summaries.")

    # Add data in batch maner, because maximum batch size is 41666
    batch_size = 16000

    for i in trange(0, len(embs), batch_size):
        l, r = i, i + batch_size

        # Add data to the collection
        collection.add(
            ids=doc_uids[l:r],
            embeddings=embs[l:r],
            metadatas=[squeeze_meta(metadatas[uid]) for uid in doc_uids[l:r]],
            documents=[summaries[uid] for uid in doc_uids[l:r]],
        )

    return persistent_client, collection

In [10]:
%%time
chroma_client, legal_practices_collection = init_legal_practices_db()

INFO:root:Start loading doc uids.
INFO:root:Successfully load doc uids.
INFO:root:Start loading embeddings.


INFO:root:Successfully load embeddings.
INFO:root:Start loading metadatas.
INFO:root:Successfully load metadatas.
INFO:root:Start loading summaries.
45465it [00:07, 6309.31it/s]
INFO:root:Successfully load summaries.


CPU times: user 5min 2s, sys: 42.1 s, total: 5min 44s
Wall time: 6min 7s


## Testing

In [11]:
chroma_client.list_collections()

[Collection(name=legal_practices), Collection(name=legal_docs)]

In [12]:
legal_practices_collection.count()

45465

In [14]:
legal_practices_collection.get().keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])

In [15]:
ids = legal_practices_collection.get()['ids'][0]
legal_practices_collection.get(ids)

{'ids': ['0l4Wt30zRKJj'],
 'embeddings': None,
 'documents': ['### Резюме судебного решения № 2-24/2024 от 15 января 2024 г. по делу № 2-24/2024\n\n#### 1. Основные сведения о деле:\n- **Регулируется:** Гражданским кодексом Российской Федерации (ГК РФ) и Гражданским процессуальным кодексом Российской Федерации (ГПК РФ).\n- **Тематика дела:** Спор о восстановлении срока для принятия наследства и признании факта принятия наследства.\n\n#### 2. Стороны и обстоятельства дела:\n- **Истец:** ФИО1.\n- **Ответчик:** Публичное акционерное общество Банк ВТБ.\n- **Обстоятельства:** ФИО1, племянница умершего ФИО5, узнала о наличии ошибок в документах, подтверждающих их родство, что помешало своевременному обращению к нотариусу для принятия наследства. У ФИО5 имелся банковский счет в Банке ВТБ с денежными средствами. Иных наследников у ФИО5 нет.\n\n#### 3. Позиции сторон:\n- **Истец:** ФИО1 ссылается на статью 1155 ГК РФ и просит восстановить срок для принятия наследства и признать факт принятия на